In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt, gc, os
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Use the [dataset from Kaggle provided by @raddar][1] and discussed in the competition's [discussion thread][2] to train an XGBoost model. 

> * originally we had 188 float/categorical type features. These were transformed into
>      * 95 np.int8/np.int16 types
>      * 93 np.float32 types
> * Most float columns with [0, 0.01] and [1, 1.01] have these values rounded up at 0 and 1 respectively. This was done to ensure no data loss, as not all features could be rounded up safely.
> * saved in parquet format (only 1.7GB training data)

[1]: https://www.kaggle.com/datasets/raddar/amex-data-integer-dtypes-parquet-format
[2]: https://www.kaggle.com/competitions/amex-default-prediction/discussion/328514

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import os
import lightgbm
import matplotlib.pyplot as plt
import seaborn as sbn
import numpy as np
import pandas as pd
import warnings
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, RocCurveDisplay, accuracy_score
import shap
import itertools

In [ ]:
import pickle, gc

In [ ]:
train = pd.read_parquet("/kaggle/input/amex-data-integer-dtypes-parquet-format/train.parquet").groupby('customer_ID').tail(4)
test = pd.read_parquet("/kaggle/input/amex-data-integer-dtypes-parquet-format/test.parquet").groupby('customer_ID').tail(4)
train_labels = pd.read_csv("../input/amex-default-prediction/train_labels.csv")
train.head()

In [ ]:
train.shape, test.shape, train_labels.shape

# Imputation

Checking Missing values having more than 40%

In [ ]:
columns = train.columns[(train.isna().sum()/len(train))*100>40]
columns

Drop the missing columns having more than 40%

In [ ]:
train = train.drop(columns, axis=1)
test = test.drop(columns, axis=1)

Fill missing values. bfill() fills missing values with the next value in the dataframe while the ffill() fills missing values with the previous value.

In [ ]:
train = train.bfill(axis='rows').ffill(axis='rows')
test = test.bfill(axis='rows').ffill(axis='rows')

In [ ]:
train.reset_index(inplace=True)
test.reset_index(inplace=True)

# Grouping

Grouping and keeping the last record of each 'customer_ID' in the dataframe, discarding all other records. Assuming that the most recent one is the most important

In [ ]:
train =train.groupby('customer_ID').tail(1)
test = test.groupby('customer_ID').tail(1)

In [ ]:
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

In [ ]:
train.shape, train_labels.shape, test.shape

Type Conversion

In [ ]:
obj_col = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
for col in obj_col:
    train[col]=train[col].astype('int').astype('str')
    test[col]=test[col].astype('int').astype('str')
    print(train[col].unique())
    print(test[col].unique())

**Data Preparation**

In [ ]:
train = train.merge(train_labels, how='inner', on="customer_ID")

In [ ]:
train.head()

In [ ]:
test_data = test.copy()
train = train.drop(['index','customer_ID', 'S_2'], axis=1)
test = test.drop(['index','customer_ID', 'S_2'], axis=1)

# Categorical Encoding

One hot Encoding for categorical features

In [ ]:
obj_col = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
train = pd.get_dummies(train, columns=obj_col, drop_first=True)
test = pd.get_dummies(test, columns=obj_col, drop_first=True)
train.shape, test.shape

In [ ]:
Features=train.loc[:, test.columns]
target = train['target']

**Model Building and Evaluation**

In [ ]:
XGB = XGBClassifier(n_estimators=300, max_depth=6, learning_rate=0.1).fit(Features, target)

In [ ]:
np.mean(cross_val_score(XGB, Features, target, scoring='accuracy', cv=5))

In [ ]:
y_pred = XGB.predict(Features)
accuracy_score(target, y_pred)

In [ ]:
cm=confusion_matrix(target, y_pred)
plt.figure(figsize=(13,5))
plt.title("Confusion Matrix")
plt.imshow(cm, alpha=0.5, cmap='PuBu')
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, cm[i, j], horizontalalignment="center")
plt.show()

In [ ]:
#AUC (ROC curve)
fpr, tpr, thresholds = roc_curve(target, y_pred)
roc_auc = auc(fpr, tpr)
display = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name=XGB)
display.plot()
plt.show()

In [ ]:
print(classification_report(target, y_pred))

In [ ]:
explainer = shap.TreeExplainer(XGB)
shap_values = explainer.shap_values(Features)
shap.summary_plot(shap_values, Features)

In [ ]:
test_data['prediction']=XGB.predict_proba(test)[:,1]
test_data[['customer_ID','prediction']].to_csv("submission.csv", index=False)